In [138]:
import os
import pymongo
import pickle
import glob
import math
import numpy
import scipy.stats
import lxml.etree as lxml
import matplotlib

path = "/home/ec2-user/SageMaker/ProjectQuigley"

os.chdir(path)

#H = House Committee
#S = Senate Committee
#J = Joint Committee

committee_dict = {}
committee_dict["H"] = {}
committee_dict["S"] = {}
committee_dict["J"] = {}
committee_dict["H"][1] = "Children and Family Law"
committee_dict["H"][2] = "Commerce and Consumer Affairs"
committee_dict["H"][3] = "Criminal Justice and Public Safety"
committee_dict["H"][4] = "Education"
committee_dict["H"][5] = "Election Law"
committee_dict["H"][6] = "Environment and Agriculture"
committee_dict["H"][7] = "Executive Departments and Administration"
committee_dict["H"][8] = "Finance"
committee_dict["H"][9] = "Finance Committee - Division I"
committee_dict["H"][10] = "Finance Committee - Division II"
committee_dict["H"][11] = "Finance Committee - Division III"
committee_dict["H"][12] = "Fish, and Game, and Marine Resources"
committee_dict["H"][13] = "Health, Human Services and Elderly Affairs"
committee_dict["H"][14] = "Judiciary"
committee_dict["H"][15] = "Labor, Industrial and Rehabilitative Services"
committee_dict["H"][16] = "Legislative Administration"
committee_dict["H"][17] = "Municipal and County Government"
committee_dict["H"][18] = "Public Works and Highways"
committee_dict["H"][19] = "Resources, Recreation and Development"
committee_dict["H"][20] = "Rules"
committee_dict["H"][21] = "Science, Technology and Energy"
committee_dict["H"][22] = "State-Federal Relations and Veterans Affairs"
committee_dict["H"][23] = "Transportation"
committee_dict["H"][24] = "Ways and Means"


committee_dict["J"][1] = "Administrative Rules"
committee_dict["J"][2] = "Legislative Ethics"


committee_dict["S"][1] = "Capital Budget"
committee_dict["S"][2] = "Commerce Committee"
committee_dict["S"][3] = "Education and Workforce Development"
committee_dict["S"][4] = "Election Law and Municipal Affairs"
committee_dict["S"][5] = "Energy and Natural Resources"
committee_dict["S"][6] = "Executive Departments and Administration"
committee_dict["S"][7] = "Finance"
committee_dict["S"][8] = "Health and Human"
committee_dict["S"][9] = "Judiciary"
committee_dict["S"][10] = "Rules and Enrolled Bills"
committee_dict["S"][11] = "Transportation"
committee_dict["S"][12] = "Ways and Means"



with open('/home/ec2-user/SageMaker/ProjectQuigley/committee_types.dict','wb') as f:
    pickle.dump(committee_dict, f)


    


In [139]:
client = pymongo.MongoClient('mongodb+srv://ridgebaseaq-docdb-dqrxk.mongodb.net/test'
                        ,username="CEDC_Quigley"
                        ,password="lpP9pCcDxQ2Y0tv8"
                        ,authSource='admin'
                        ,authMechanism='SCRAM-SHA-1')

database = client['legislative_documents']
collection = database['legiscan_bill_metadata']
times = [2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]
bill_sponsor_table = {}
for t in times:
    bill_sponsor_table[t] = {}

for b in collection.find({'state': "NH"}):
    bill_sponsor_table[b['session']['year_start']][b['bill_id']] = {'primary': [], 'co': [], 'other': []}
    for s in b['sponsors']:
        if s['sponsor_type_id'] == 1:
            bill_sponsor_table[b['session']['year_start']][b['bill_id']]['primary'].append(s['people_id'])
        elif s['sponsor_type_id'] == 2:
            bill_sponsor_table[b['session']['year_start']][b['bill_id']]['co'].append(s['people_id'])
        else:
            bill_sponsor_table[b['session']['year_start']][b['bill_id']]['other'].append(s['people_id'])
            
       


{'primary': [10918], 'co': [4871, 4949]}


In [140]:
people_table = {}

client = pymongo.MongoClient('mongodb+srv://ridgebaseaq-docdb-dqrxk.mongodb.net/test'
                        ,username="CEDC_Quigley"
                        ,password="lpP9pCcDxQ2Y0tv8"
                        ,authSource='admin'
                        ,authMechanism='SCRAM-SHA-1')

database = client['legislative_documents']
collection = database['legiscan_people_data']

chamber_ref = {1: "H", 2: "S"}
for person in collection.find():
    people_table[person['people_id']] = {'party_id': int(person['party_id']), 'state_id': int(person['state_id']), 'district': person['district'], 'chamber': chamber_ref[person['role_id']]}   

print(people_table[10500])
with open('/home/ec2-user/SageMaker/ProjectQuigley/people_info_table.dict','wb') as f:
    pickle.dump(people_table, f)


{'party_id': 2, 'state_id': 29, 'district': 'SD-023', 'chamber': 'S'}


In [141]:

total_roll = []


times = [2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]
states = ['NH']

roll_call_by_year = {}
for s in states:
    roll_call_by_year[s] = {}
    for t in times:
        roll_call_by_year[s][t] = []

    
    

#print(roll_call_by_year['CA'][2015])

client = pymongo.MongoClient('mongodb+srv://ridgebaseaq-docdb-dqrxk.mongodb.net/test'
                        ,username="CEDC_Quigley"
                        ,password="lpP9pCcDxQ2Y0tv8"
                        ,authSource='admin'
                        ,authMechanism='SCRAM-SHA-1')
database = client['legislative_documents']
collection = database['legiscan_bill_metadata']
#{"$and": [{'state': {'$in': states}}, {'session.year_start': {'$in': times}}]}

for l in collection.find({"$and": [{'state': {'$in': states}}, {'session.year_start': {'$in': times}}]}):
    for v in l['votes']:
        roll_call_by_year[l['state']][int(l["session"]["year_start"])].append(v['roll_call_id'])




print(len(roll_call_by_year['NH'][2015]))
#print(len(roll_call_by_year['CA'][2015]))
#print(missing_data)

#print(roll_call_by_year['NH'][2010][3])
#278
#404321

278


In [142]:
# create a matrix by collecting all members of house or senate from bills



times = [2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]
states = ['NH']
chamber = ["H", "S"]
politicians_by_year = {}


for s in states:
    politicians_by_year[s] = {}
    for t in times:
        politicians_by_year[s][t] = {}
        for c in chamber:
            politicians_by_year[s][t][c] = []

            




client = pymongo.MongoClient('mongodb+srv://ridgebaseaq-docdb-dqrxk.mongodb.net/test'
                        ,username="CEDC_Quigley"
                        ,password="lpP9pCcDxQ2Y0tv8"
                        ,authSource='admin'
                        ,authMechanism='SCRAM-SHA-1')
database = client['legislative_documents']
collections = database['legiscan_vote_data']


counter = 0
h_counter = 0
error_list = []


c = 0
# {"$and": [{'state': {'$in': ['AK']}}, {'session.year_start': {'$in': [2016]}}]}
# {"$and": [{'state': {'$in': ['NH']}}, {'session.year_start': {'$in': [2010, 2011]}}]}
#for item in collection.find({"date" : {"$in": mylist}}):
for s in states:
    for t in times:
        for i in collections.find({'roll_call_id': {'$in':  roll_call_by_year[s][t]}}):
            for j in i['votes']:
                person = j['people_id']
                if person in politicians_by_year[s][t][i['chamber']]:
                    continue
                else:
                    politicians_by_year[s][t][i['chamber']].append(j['people_id'])
                    
            """if i['chamber'] == "H":
                for j in i['votes']:
                    person = j['people_id']
                    if person in politicians_by_year[s][t][i['chamber']]:
                        continue
                    else:
                        politicians_by_year[s][t][i['chamber']].append(j['people_id'])
            elif i['chamber'] == "S":
                for j in i['votes']:
                    person = j['people_id']
                    if person in politicians_by_year[s][t][i['chamber']]:
                        continue
                    else:  
                         politicians_by_year[s][t][i['chamber']].append(j['people_id'])
            else:
                print("error")
            """



print(len(politicians_by_year['NH'][2015]["S"]))
#24


#print(error_list)

print(len(politicians_by_year['NH'][2015]["H"]))


print(politicians_by_year['NH'][2015]["S"])



with open('/home/ec2-user/SageMaker/ProjectQuigley/politicians_by_year.dict', 'wb') as f:
    pickle.dump(politicians_by_year, f)
#402



24
402
[4760, 4775, 4783, 4787, 4789, 4792, 4814, 4821, 4871, 4926, 4953, 4958, 10455, 10462, 10471, 10497, 10500, 11579, 11989, 14489, 15124, 15125, 16397, 16619]


In [143]:
times = [2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]
states = ['NH']
chamber = ["H", "S"]
people_party_table = {}
people_district_table = {}

for s in states:
    people_party_table[s] = {}
    people_district_table[s] = {}
    for t in times:
        people_party_table[s][t] = {}
        people_district_table[s][t] = {}
        for c in chamber:
            people_party_table[s][t][c] = {}
            people_district_table[s][t][c] = {}
            


client = pymongo.MongoClient('mongodb+srv://ridgebaseaq-docdb-dqrxk.mongodb.net/test'
                        ,username="CEDC_Quigley"
                        ,password="lpP9pCcDxQ2Y0tv8"
                        ,authSource='admin'
                        ,authMechanism='SCRAM-SHA-1')
database = client['legislative_documents']
collection = database['legiscan_people_data']

            
for s in states:
    for t in times:
        for c in chamber:
            for l in collection.find({'people_id': {'$in': politicians_by_year[s][t][c]}}):
                #person = l['people_id']
                people_party_table[s][t][c][l['people_id']] = l['party_id']
                people_district_table[s][t][c][l['people_id']] = l['district']
                

print(people_party_table["NH"][2015]["S"][10500])
print(people_district_table["NH"][2015]["S"][10500])

with open('/home/ec2-user/SageMaker/ProjectQuigley/people_party_table.dict','wb') as f:
    pickle.dump(people_party_table, f)
    
with open('/home/ec2-user/SageMaker/ProjectQuigley/people_district_table.dict','wb') as f:
    pickle.dump(people_district_table, f)
    


2
SD-023


In [238]:
pol_party = {}
client = pymongo.MongoClient('mongodb+srv://ridgebaseaq-docdb-dqrxk.mongodb.net/test'
                        ,username="CEDC_Quigley"
                        ,password="lpP9pCcDxQ2Y0tv8"
                        ,authSource='admin'
                        ,authMechanism='SCRAM-SHA-1')
database = client['legislative_documents']
collection = database['legiscan_people_data']
for l in collection.find():
    pol_party[l['people_id']] = int(l['party_id'])
print(pol_party[12869])


2


In [299]:
# leadership/ideology matrices for a house or senate bill
import numpy as np
import pandas as pd

state_and_year_matrices = {}

times = [2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]
states = ['NH']
state_and_year_matrices = {}
for s in states:
    
    state_and_year_matrices[s] = {}
    
    for t in times:
        
        pos_H = len(politicians_by_year[s][t]['H'])
        pos_S = len(politicians_by_year[s][t]['S'])
        politicians_by_year[s][t]['H'].sort()
        politicians_by_year[s][t]['S'].sort()
        state_and_year_matrices[s][t] = {}
        state_and_year_matrices[s][t] = {"H": numpy.identity(pos_H, numpy.float), "S": numpy.identity(pos_S, numpy.float), "H_ind": {}, "S_ind": {}}


        index_to_id_H = {}
        id_to_index_H = {}
        for i, val in enumerate(politicians_by_year[s][t]['H']):
            index_to_id_H[i] = int(val)
            id_to_index_H[int(val)] = i
      

        index_to_id_S = {}
        id_to_index_S = {}
        for j, vval in enumerate(politicians_by_year[s][t]['S']):
            index_to_id_S[j] = int(vval)
            id_to_index_S[int(vval)] = j


        client = pymongo.MongoClient('mongodb+srv://ridgebaseaq-docdb-dqrxk.mongodb.net/test'
                                ,username="CEDC_Quigley"
                                ,password="lpP9pCcDxQ2Y0tv8"
                                ,authSource='admin'
                                ,authMechanism='SCRAM-SHA-1')
        database = client['legislative_documents']
        collection = database['legiscan_bill_metadata']

        for bl in collection.find({"$and": [{'state': s}, {'session.year_start': t}]}):
            if bl['state'] == s and bl['session']['year_start'] == t:
                current_prim_S = []
                current_co_S = []
                current_prim_H = []
                current_co_H = []

                if bl['body'] == 'H':
                    for sp in bl['sponsors']:
                    #mat = state_and_year_matrices[s][t]['H']

                        if not sp["people_id"] in id_to_index_H.keys():

                            b = numpy.identity(pos_H+1, numpy.float)
                            b[:-1,:-1] = state_and_year_matrices[s][t]['H']
                            state_and_year_matrices[s][t]['H'] = b
                            id_to_index_H[sp["people_id"]] = pos_H
                            index_to_id_H[pos_H] = sp["people_id"]
                            pos_H += 1
                    for sp in bl['sponsors']:
                        if sp["sponsor_type_id"] == 1:

                            current_prim_H.append(sp["people_id"])

                        elif sp["sponsor_type_id"] == 2:

                            current_co_H.append(sp["people_id"])
                           
                        else:
                            continue
                    for row in current_prim_H:
                        for col in current_co_H:
                            state_and_year_matrices[s][t]['H'][id_to_index_H[row], id_to_index_H[col]] += 1
                            

                elif bl['body'] == 'S':
                    for sp in bl['sponsors']:


                        if not sp["people_id"] in id_to_index_S.keys():

                            #b = np.zeros((pos_S+1, pos_S+1))
                            b = numpy.identity(pos_S+1, numpy.float)
                            b[:-1,:-1] = state_and_year_matrices[s][t]['S']
                            state_and_year_matrices[s][t]['S'] = b
                            id_to_index_S[sp["people_id"]] = pos_S
                            index_to_id_S[pos_S] = sp["people_id"]

                            pos_S += 1
                    for sp in bl['sponsors']:
                        if sp["sponsor_type_id"] == 1:

                            current_prim_S.append(sp["people_id"])

                        elif sp["sponsor_type_id"] == 2:

                            current_co_S.append(sp["people_id"])
                           
                        else:
                            continue
                    for row in current_prim_S:
                        for col in current_co_S:
                            state_and_year_matrices[s][t]['S'][id_to_index_S[row], id_to_index_S[col]] += 1
                            
                state_and_year_matrices[s][t]["H_ind"] = index_to_id_H
                state_and_year_matrices[s][t]["S_ind"] = index_to_id_S

with open('/home/ec2-user/SageMaker/ProjectQuigley/sponsor_matrix.dict','wb') as f:
    pickle.dump(state_and_year_matrices, f)


In [300]:

def onenorm(u):

    return sum(abs(u))
def smooth_matrix(nreps, P):

    for i in range(nreps):
        for j in range(nreps):
            P[i,j] = math.sqrt(P[i,j])

def onenorm(u):
    return sum(abs(u))

def rescale(u, log=False):

    u = (u - min(u)) / (max(u) - min(u))


    if log:
        m = numpy.median(u)
        s = -m**2/(2*m - 1)
        if s > 0:
            u = numpy.log(u + s)
            u = (u - min(u)) / (max(u) - min(u))
    return [float(v) for v in u]

def leadership_analysis(state, year, chamber):
    
    P = state_and_year_matrices[state][year][chamber]
    
    nreps = state_and_year_matrices[state][year][chamber].shape[0]
    for col in range(nreps):
        s = sum(P[:,col])
        if s == 0:
            #raise ValueError()
            #print("Error")
            pass
        if s < 10:
            P[:,col] += (10.0-s)/nreps
            s = 10
        P[:,col] = P[:,col] / s
    v = numpy.ones( (nreps, 1) ) / float(nreps)
    c = 0.85
    x = numpy.ones( (nreps, 1) ) / float(nreps)
    
    while True:
        y = c * numpy.dot(P, x)
        w = onenorm(x) - onenorm(y)
        y = y + w*v
        err = onenorm(y-x)
        if err < .00000000001:
            break
        x = y

    x = rescale(x, log=True)

    return x # this is the pagerank

def ideology_analysis(state, year, chamber):
    P = state_and_year_matrices[state][year][chamber]
    nreps = state_and_year_matrices[state][year][chamber].shape[0]

    if chamber == "H":
        
        u, s, vh = numpy.linalg.svd(P)
        spectrum = vh[1,:]
        
        R_scores = [spectrum[i] for i in range(nreps) if pol_party[state_and_year_matrices[state][year]['H_ind'][i]] == 2]
        R_score_mean = sum(R_scores)/len(R_scores)
        spectrum = spectrum * R_score_mean

        spectrum = rescale(spectrum)
        
        return spectrum
    
    if chamber == "S":
        
        u, s, vh = numpy.linalg.svd(P)
        spectrum = vh[1,:]
        
        R_scores = [spectrum[i] for i in range(nreps) if pol_party[state_and_year_matrices[state][year]['S_ind'][i]] == 2]
        R_score_mean = sum(R_scores)/len(R_scores)
        spectrum = spectrum * R_score_mean


        spectrum = rescale(spectrum)
        
        return spectrum
        


In [332]:

leadership_table = {'NH': {}}
times = [2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]

for t in times:
    leadership_table['NH'][t] = {'H': {}, 'S': {}}
 
    
    
    lead_H = leadership_analysis("NH", t, "H")
    for v in range(len(lead_H)):
        leadership_table['NH'][t]['H'][state_and_year_matrices['NH'][t]['H_ind'][v]]=lead_H[v]
        
    
    lead_S = leadership_analysis("NH", t, "S")
    for vv in range(len(lead_S)):
        leadership_table['NH'][t]['S'][state_and_year_matrices['NH'][t]['S_ind'][vv]]=lead_S[vv]


with open('/home/ec2-user/SageMaker/ProjectQuigley/leadership_table.dict','wb') as f:
    pickle.dump(leadership_table, f)



/home/ec2-user/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/numpy/lib/function_base.py:4033: RuntimeWarning: Invalid value encountered in median
  r = func(a, **kwargs)


In [339]:
import operator
# functions for time intervals of leadership/ideology
def leadership_analysis_v2(nreps, P, list_of_id):
    for col in range(nreps):
        s = sum(P[:,col])
        if s == 0:
            
            #print("Error")
            pass
        if s < 10:
            P[:,col] += (10.0-s)/nreps
            s = 10
        P[:,col] = P[:,col] / s
    v = numpy.ones( (nreps, 1) ) / float(nreps)
    c = 0.85
    x = numpy.ones( (nreps, 1) ) / float(nreps)
    
    while True:
        y = c * numpy.dot(P, x)
        w = onenorm(x) - onenorm(y)
        y = y + w*v
        err = onenorm(y-x)
        if err < .00000000001:
            break
        x = y

    x = rescale(x, log=True)

    if len(x) != len(list_of_id):
        print("ERROR in leadership")
    
    res = {}
    
    for i in range(len(x)):
        res[list_of_id[i]] = x[i]

    ret = dict( sorted(res.items(), key=operator.itemgetter(1),reverse=True))
    return ret

def ideology_analysis_v2(nreps, P, list_of_id, party_list):
    u, s, vh = numpy.linalg.svd(P)
    spectrum = vh[1,:]

    R_scores = [spectrum[i] for i in range(nreps) if party_list[i] == 2]
    R_score_mean = sum(R_scores)/len(R_scores)
    spectrum = spectrum * R_score_mean

    spectrum = rescale(spectrum)
    
    res = {}
    
    for i in range(len(spectrum)):
        res[list_of_id[i]] = spectrum[i]
    if len(spectrum) != len(list_of_id):
        print("ERROR in ideology")
    ret = {}
    ret = dict(sorted(res.items(), key=operator.itemgetter(1),reverse=True))
    return ret

In [340]:
# data wrangling for time intervals

sponsor_table = {}
temp_sponsor_table = {}

years = [2015, 2016, 2017, 2018, 2019]
client = pymongo.MongoClient('mongodb+srv://ridgebaseaq-docdb-dqrxk.mongodb.net/test'
                        ,username="CEDC_Quigley"
                        ,password="lpP9pCcDxQ2Y0tv8"
                        ,authSource='admin'
                        ,authMechanism='SCRAM-SHA-1')
database = client['legislative_documents']
collection = database['legiscan_bill_metadata']

for t in years:
    temp_sponsor_table[t] ={'list_of_sponsors': []}
    sponsor_table[t] = {}
    client = pymongo.MongoClient('mongodb+srv://ridgebaseaq-docdb-dqrxk.mongodb.net/test'
                        ,username="CEDC_Quigley"
                        ,password="lpP9pCcDxQ2Y0tv8"
                        ,authSource='admin'
                        ,authMechanism='SCRAM-SHA-1')
    database = client['legislative_documents']
    collection = database['legiscan_bill_metadata']

    for bill in collection.find({"$and": [{'state': 'NH'}, {'session.year_start': t}]}):
            sponsor_table[t][bill['bill_id']]={"primary": [], "co": [], "other": []}
            
            for s in bill['sponsors']:
                
                if s['sponsor_type_id'] == 1:
                    
                    sponsor_table[t][bill['bill_id']]['primary'].append(int(s['people_id']))
                
                    if not s['people_id'] in temp_sponsor_table[t]['list_of_sponsors']:
                        temp_sponsor_table[t]['list_of_sponsors'].append(int(s['people_id']))
                    
                    
                elif s['sponsor_type_id'] == 2:
                    sponsor_table[t][bill['bill_id']]['co'].append(int(s['people_id']))
                    
                    if not s['people_id'] in temp_sponsor_table[t]['list_of_sponsors']:
                        temp_sponsor_table[t]['list_of_sponsors'].append(int(s['people_id']))
                else:
                    sponsor_table[t][bill['bill_id']]['other'].append(int(s['people_id']))





In [341]:
# matrix construction for time intervals
def build_matrix(list_of_years):
    res = {}
    total_list_of_sponsors = []
    for t in list_of_years:
        for person in temp_sponsor_table[t]['list_of_sponsors']:
            if not person in total_list_of_sponsors:
                total_list_of_sponsors.append(int(person))


    total_list_of_sponsors.sort()
    
 

    nxn = len(total_list_of_sponsors)

    _P = numpy.identity(nxn, numpy.float)

    res = {}
    ordered_id = []
    ordered_party = []
    
    for i, val in enumerate(total_list_of_sponsors): 
        res[val] = {"index": i, "party": pol_party[val]}
        ordered_id.append(val)
        ordered_party.append(pol_party[val])


    for t in list_of_years:
        for bill in sponsor_table[t]:  
            if sponsor_table[t][bill]['co'] != []:
                for row in sponsor_table[t][bill]['primary']:
                    for col in sponsor_table[t][bill]['co']:
                        _P[res[row]['index'], res[col]['index']] += 1.0

    return nxn, _P, ordered_id, ordered_party

        
    

In [342]:
num, matr, list_id, list_party = build_matrix([2015, 2016, 2017, 2018, 2019])

res1 = leadership_analysis_v2(num, matr, list_id)

res2 = ideology_analysis_v2(num, matr, list_id, list_party)


with open('/home/ec2-user/SageMaker/ProjectQuigley/politician_leadership.dict', 'wb') as f:
    pickle.dump(res1, f)
with open('/home/ec2-user/SageMaker/ProjectQuigley/politician_ideology.dict', 'wb') as f:
    pickle.dump(res2, f)
#print(res2)

In [310]:
#leadership by district

total_distr = []
for key in people_table.keys():
    if not people_table[key]['district'] in total_distr:
        total_distr.append(people_table[key])
sen = []
house = []
for key in people_table.keys():
    if not (people_table[key]['district'] in sen) and (people_table[key]['chamber'] == "S"):
        sen.append(people_table[key])
        
for key in people_table.keys():
    if not (people_table[key]['district'] in sen) and (people_table[key]['chamber'] == "H"):
        house.append(people_table[key])




40
697
737


In [327]:
# data wrangling for district and time

d_sponsor_table = {}
d_temp_sponsor_table = {}

years = [2015, 2016, 2017, 2018, 2019]
client = pymongo.MongoClient('mongodb+srv://ridgebaseaq-docdb-dqrxk.mongodb.net/test'
                        ,username="CEDC_Quigley"
                        ,password="lpP9pCcDxQ2Y0tv8"
                        ,authSource='admin'
                        ,authMechanism='SCRAM-SHA-1')
database = client['legislative_documents']
collection = database['legiscan_bill_metadata']

for t in years:
    
    d_temp_sponsor_table[t] ={'list_of_sponsors': []}
    d_sponsor_table[t] = {}
    
    client = pymongo.MongoClient('mongodb+srv://ridgebaseaq-docdb-dqrxk.mongodb.net/test'
                        ,username="CEDC_Quigley"
                        ,password="lpP9pCcDxQ2Y0tv8"
                        ,authSource='admin'
                        ,authMechanism='SCRAM-SHA-1')
    database = client['legislative_documents']
    collection = database['legiscan_bill_metadata']

    for bill in collection.find({"$and": [{'state': 'NH'}, {'session.year_start': t}]}):
            d_sponsor_table[t][bill['bill_id']]={"primary": [], "co": [], "other": []}
            
            for s in bill['sponsors']:
                
                if s['sponsor_type_id'] == 1:
                    
                    d_sponsor_table[t][bill['bill_id']]['primary'].append(s['district'])
                
                    if not s['district'] in d_temp_sponsor_table[t]['list_of_sponsors']:
                        d_temp_sponsor_table[t]['list_of_sponsors'].append(s['district'])
                    
                    
                elif s['sponsor_type_id'] == 2:
                    d_sponsor_table[t][bill['bill_id']]['co'].append(s['district'])
                    
                    if not s['district'] in d_temp_sponsor_table[t]['list_of_sponsors']:
                        d_temp_sponsor_table[t]['list_of_sponsors'].append(s['district'])
                else:
                    d_sponsor_table[t][bill['bill_id']]['other'].append(s['district'])





In [330]:
# matrix construction for time intervals and district
def d_build_matrix(list_of_years):
    res = {}
    total_list_of_sponsors = []
    for t in list_of_years:
        for district in d_temp_sponsor_table[t]['list_of_sponsors']:
            if not district in total_list_of_sponsors:
                total_list_of_sponsors.append(district)


    total_list_of_sponsors.sort()
    
 

    nxn = len(total_list_of_sponsors)

    _P = numpy.identity(nxn, numpy.float)

    res = {}
    ordered_id = []
    
    
    for i, val in enumerate(total_list_of_sponsors): 
        res[val] = {"index": i}
        ordered_id.append(val)
    

    for t in list_of_years:
        for bill in d_sponsor_table[t]:
            if d_sponsor_table[t][bill]['co'] != []:
                for row in d_sponsor_table[t][bill]['primary']:
                    for col in d_sponsor_table[t][bill]['co']:
                        _P[res[row]['index'], res[col]['index']] += 1.0

    return nxn, _P, ordered_id

        
    

In [351]:

def determine_person_ideology(person):
    """
    if we dont have a person's ideology i.e they never sponsored a bill
    do the following two possible 2 possible implementations:
    1)
        1: go through all bills and look at how the person voted
        2: then take the average ideology score of the members that voted the same as the person
    2)
        same as 1) but take the average ideology score of the members in the person's party that had the same vote

        for example:

            person_id voted Yea on a bill
            person_id is R
            average all R that voted Yea
    """
    pass


def determine_district_idelogy(district):
    """
    2 methods of implementation:
    
    1)
        ~using ideology instead of political party~
        Once all politicians have a ideology score
        Go through all districts by year
        Average the ideology score
        Save to dictionary for that year

        for year intervals, average the years in the interval ie 2015, 2016, 2017

        then we have the ideology number

        if ideology number > 0.5 then 1
        
    2)
        same as 1) but take the average of R and D in district
        R = 1 and D = 0
        if average > 0.5 -> district = R
        if average < 0.5 -> district = D
        if average = 0.5 -> district = 0.5 or Tie
    """
    pass

"""
# for independents, score according to the whole chamber
        ss_20 = scipy.stats.scoreatpercentile(spectrum, 20)
        ss_80 = scipy.stats.scoreatpercentile(spectrum, 80)

        pp_20 = scipy.stats.scoreatpercentile(pagerank, 20)
        pp_80 = scipy.stats.scoreatpercentile(pagerank, 80)
        
            descr_table = [ # no damn commas
        ["left-leaning Independent leader", "moderate Independent leader", "right-leaning Independent leader"],
        ["left-leaning Independent", "centrist Independent", "right-leaning Independent"],
        ["lonely left-leaning Independent", "lonely centrist Independent", "lonely right-leaning Independent"],
        ]
"""

'\n# for independents, score according to the whole chamber\n\t\t\tss_20 = scipy.stats.scoreatpercentile(spectrum, 20)\n\t\t\tss_80 = scipy.stats.scoreatpercentile(spectrum, 80)\n\t\t\t\n\t\t\tpp_20 = scipy.stats.scoreatpercentile(pagerank, 20)\n\t\t\tpp_80 = scipy.stats.scoreatpercentile(pagerank, 80)\n'

In [369]:
district_leadership = {}
people_leadership = {}
people_ideology = {}

years = [[2015], [2016], [2017], [2018], [2019], [2015, 2016], [2016, 2017], [2017, 2018], [2018, 2019],[2015, 2016, 2017], [2016, 2017, 2018], [2017, 2018, 2019], [2015, 2016, 2017, 2018], [2016, 2017, 2018, 2019], [2015, 2016, 2017, 2018, 2019]]

for time in years:
    key = ""
    for i in range(len(time)):
        if i == len(time) - 1:
            key += str(time[i])
        else:
            key += str(time[i]) + "-"
        
    #build district leadership
    d_numrows, D_mat, d_order = d_build_matrix(time)
    
    district_leadership[key] = leadership_analysis_v2(d_numrows, D_mat, d_order)
    
    #build people leadership
    numrows, P_mat, order, party = build_matrix(time)
    people_leadership[key] = leadership_analysis_v2(numrows, P_mat, order)
    
    #build people ideology
    people_ideology[key] = ideology_analysis_v2(numrows, P_mat, order, party)
    


In [370]:


with open('/home/ec2-user/SageMaker/ProjectQuigley/district_leadership.dict', 'wb') as f:
    pickle.dump(district_leadership, f)
with open('/home/ec2-user/SageMaker/ProjectQuigley/people_leadership.dict', 'wb') as f:
    pickle.dump(people_leadership, f)
with open('/home/ec2-user/SageMaker/ProjectQuigley/people_ideology.dict', 'wb') as f:
    pickle.dump(people_ideology, f)


In [371]:
summer = 0
for key in people_ideology["2016-2017-2018-2019"].keys():
    summer += people_ideology["2016-2017-2018-2019"][key]
summer/len(people_ideology["2016-2017-2018-2019"])
#print(people_ideology["2017-2018-2019"])

0.5605690451344026